# Model value comparison

In [1]:
%load_ext autoreload

In [2]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [3]:
BASE_PATH = os.path.join(os.getcwd(), "..", "..")
MODULES_PATH = os.path.join(BASE_PATH, "modules")
DATASET_PATH = os.path.join(BASE_PATH, "datasets")

In [4]:
sys.path.append(MODULES_PATH)

In [5]:
from active_learning import TrainConfig
from data import BenchmarkData, DataSetType
from bayesian import McDropout, MomentPropagation
from models import FcholletCNN, setup_growth

# Load data

In [6]:
num_classes = 5
mnist = BenchmarkData(DataSetType.MNIST, os.path.join(DATASET_PATH, "mnist"), dtype=np.float, classes=num_classes)

In [7]:
print("Inputs-Shape: {}".format(mnist.inputs.shape))
print("Targets-Shape: {}".format(mnist.targets.shape))

Inputs-Shape: (35735, 28, 28, 1)
Targets-Shape: (35735,)


In [8]:
x_train, x_test, y_train, y_test = train_test_split(mnist.inputs, mnist.targets)

# Train both models

In [9]:
setup_growth()

1 Physical GPU's,  1 Logical GPU's


In [10]:
base_model = FcholletCNN(num_classes)
base_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [11]:
base_model.fit(x_train, y_train, epochs=100, batch_size=60)

Epoch 1/100
447/447 [==============================] - 8s 18ms/step - loss: 0.6926 - accuracy: 0.9453
Epoch 2/100
447/447 [==============================] - 2s 4ms/step - loss: 0.0636 - accuracy: 0.9799
Epoch 3/100
447/447 [==============================] - 2s 4ms/step - loss: 0.0461 - accuracy: 0.9868
Epoch 4/100
447/447 [==============================] - 2s 4ms/step - loss: 0.0396 - accuracy: 0.9880
Epoch 5/100
447/447 [==============================] - 2s 5ms/step - loss: 0.0357 - accuracy: 0.9907
Epoch 6/100
447/447 [==============================] - 2s 5ms/step - loss: 0.0281 - accuracy: 0.9920
Epoch 7/100
447/447 [==============================] - 2s 4ms/step - loss: 0.0250 - accuracy: 0.9929
Epoch 8/100
447/447 [==============================] - 2s 4ms/step - loss: 0.0267 - accuracy: 0.9922
Epoch 9/100
447/447 [==============================] - 2s 4ms/step - loss: 0.0245 - accuracy: 0.9926
Epoch 10/100
447/447 [==============================] - 2s 4ms/step - loss: 0.0206 - accur

447/447 [==============================] - 2s 4ms/step - loss: 0.0105 - accuracy: 0.9977
Epoch 82/100
447/447 [==============================] - 2s 5ms/step - loss: 0.0126 - accuracy: 0.9978
Epoch 83/100
447/447 [==============================] - 2s 4ms/step - loss: 0.0080 - accuracy: 0.9986
Epoch 84/100
447/447 [==============================] - 2s 4ms/step - loss: 0.0062 - accuracy: 0.9987
Epoch 85/100
447/447 [==============================] - 2s 4ms/step - loss: 0.0081 - accuracy: 0.9986
Epoch 86/100
447/447 [==============================] - 2s 4ms/step - loss: 0.0049 - accuracy: 0.9988
Epoch 87/100
447/447 [==============================] - 2s 4ms/step - loss: 0.0122 - accuracy: 0.9986
Epoch 88/100
447/447 [==============================] - 2s 4ms/step - loss: 0.0097 - accuracy: 0.9982
Epoch 89/100
447/447 [==============================] - 2s 4ms/step - loss: 0.0078 - accuracy: 0.9985
Epoch 90/100
447/447 [==============================] - 2s 5ms/step - loss: 0.0125 - accuracy: 

In [19]:
base_model.evaluate(x_test, y_test)

280/280 [==============================] - 1s 2ms/step - loss: 0.0533 - accuracy: 0.9962


[0.053324561566114426, 0.9961943030357361]

In [44]:
base_model.predict(x_test, batch_size=10)

array([[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.0000000e+00,
        0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        1.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.0000000e+00,
        0.0000000e+00],
       ...,
       [0.0000000e+00, 1.0000000e+00, 1.9358103e-38, 0.0000000e+00,
        0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.0000000e+00,
        0.0000000e+00]], dtype=float32)

In [14]:
# McDropout model
%autoreload 2
mc_dropout = McDropout(base_model, TrainConfig())
mc_dropout.evaluate(x_test, y_test, batch_size=60, n_times=1)

TypeError: call() got an unexpected keyword argument 'batch_size'

In [193]:
result = mc_dropout.batch_prediction(x_test, n_times=1, batch_size=60)

for i in result:
    print(i)
    break

TypeError: call() got an unexpected keyword argument 'batch_size'

In [178]:
np.mean(result_labels==y_test)

0.9959704499664204